In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/home/vicky/Downloads/NLSeer/NLSeer/finalized_df_cleaned.csv")

In [3]:
print(df.head)

<bound method NDFrame.head of          ACC                                       AnnotEncoded  \
0     O75439  3333333333333333333333333333333333333333333330...   
1     Q2TBK2  3333333333333333333333333333333333333333333333...   
2     Q5VY80  0000000000000000000000000000000000000000000000...   
3     Q9BZM6  0000000000000000000000000000000000000000000000...   
4     O75489  3333333333333333333333333333333333330000000000...   
...      ...                                                ...   
2947  Q96CK0  0000000000000000000000000000000000000000000000...   
2948  Q96CK0  0000000000000000000000000000000000000000000000...   
2949  Q24JY4  0000000000000000000000000000000000000066666666...   
2950  O43257  0000000000000000000000000000000000000066666666...   
2951  Q8R331  0000000000000000000000000000000000000066666666...   

                                               Sequence Types  Length  
0     MAAAAARVVLSSAARRRLWGFSESLLIRGAAGRSLYFGENRLRSTQ...    MT     489  
1     MAAAAFAVPRGVQLR

In [4]:
# Preprocessing function
def preprocess_sequences(df, max_length):
    char_set = {'A', 'C', 'G', 'T', 'D','E','F', 'H','I', 'K','L','M','N', 'P','Q', 'R','S','V','W','Y'}
    char_to_int = {char: i for i, char in enumerate(char_set)}
    sequences = []
    for seq in df['Sequence']:
        # Truncate or pad sequences
        seq = seq[:max_length].ljust(max_length, '0')
        # Convert to one-hot encoding
        one_hot_seq = [char_to_int[char] for char in seq]
        sequences.append(one_hot_seq)
    return torch.tensor(sequences, dtype=torch.float)


In [5]:
char_set = {'A', 'C', 'D','E','F', 'G', 'H','I', 'K','L','M','N', 'P','Q', 'R','S', 'T','V','W','Y'}
len(char_set)

20

In [29]:
sequence = df['Sequence'][1]
max_length = 1000
seq = sequence[:max_length].ljust(max_length, '0')

In [24]:
seq

'MAAAAFAVPRGVQLRVLTERLLRGGVRELLRPRLSGSTPGSERDFSLSHSRGTVIVERWWKVPLAGEGRKPRLHRRHRVYKLVEDTKHRPKENLELILTQSVDELGVRGDLVSVKKSVGRNRLLPEGLAVYASPENKKLFEEEKLLRQEGKLDKIQTKAGEATVKFLRSCHLEVGMKNNVKWELNPEIVARHFLRNLGVVVAPHALKLPEEPITQRGEYWCEVTVNGLDTVRVPMSVVNFERPKTKRYKYWLAQQAAKGMASTSFQKI00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [31]:
sequence = df['Sequence']
max_length = 1000
# X = preprocess_sequences(df, max_length)

In [6]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        output = self.fc(lstm_out.view(len(input_seq), -1))
        output = self.sigmoid(output)
        return output[-1]

In [7]:
# reading the encoded data by Ming
df_encoded = pd.read_csv("/home/vicky/Downloads/NLSeer/NLSeer/cnn_input_data.csv")

In [8]:
df_encoded["Encoding"][1]

'[[ -0.          -0.          -0.         ...  -0.          -0.\n   -0.        ]\n [-17.09913826  -0.          -0.         ...  -0.          -0.\n   -0.        ]\n [-12.31542206  -0.          -0.         ...  -0.          -0.\n   -0.        ]\n ...\n [  0.           0.           0.         ...   0.           0.\n    0.        ]\n [  0.           0.           0.         ...   0.           0.\n    0.        ]\n [  0.           0.           0.         ...   0.           0.\n    0.        ]]'

In [9]:
# Using updated data by Ming
import pickle

# Specify the path to the pickled file
file_path = '/home/vicky/Downloads/NLSeer/NLSeer/testing_data.pkl'

# Open the pickled file in read mode
with open(file_path, 'rb') as f:
    # Load the pickled object
    loaded_df = pickle.load(f)

# Now you can use the loaded_object as needed
print(loaded_df)

                                               Encoding  Label
0     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
1     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
2     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
3     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
4     [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      1
...                                                 ...    ...
2709  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2710  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2711  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2712  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0
2713  [[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...      0

[2714 rows x 2 columns]


In [10]:
X = torch.tensor(loaded_df["Encoding"])
y = loaded_df["Label"]

/tmp/ipykernel_150286/616263396.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  X = torch.tensor(loaded_df["Encoding"])


In [15]:
X.shape


torch.Size([2714, 1000, 20])

In [12]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert Pandas DataFrame columns to PyTorch tensors
X_train_tensor = torch.tensor(X_train.tolist(), dtype=torch.float)
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.float)
X_test_tensor = torch.tensor(X_test.tolist(), dtype=torch.float)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.float)

# Define DataLoader for train and test sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [17]:
X_train_tensor.shape


torch.Size([2171, 1000, 20])

In [16]:
for X_batch, y_batch in train_dataset:
    print("X_batch shape:", X_batch.shape)
    print("y_batch shape:", y_batch.shape)
    break

X_batch shape: torch.Size([1000, 20])
y_batch shape: torch.Size([])


In [18]:
# Set parameters
max_length = 1000
input_size = 20000  # Number of characters in the character set
hidden_size = 128
output_size = 1

In [20]:
# Initialize model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in train_dataloader:
        # Move data to GPU
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on test set
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for batch_X, batch_y in test_dataloader:
        # Move data to GPU
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        output = model(batch_X)
        test_loss += criterion(output, batch_y).item()
        predicted = torch.round(output)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f'Test Accuracy: {(100 * correct / total):.2f}%')

Epoch [1/10], Loss: 1.5479
Epoch [2/10], Loss: 0.2142
Epoch [3/10], Loss: 0.0004
Epoch [4/10], Loss: 0.0003
Epoch [5/10], Loss: 0.0001
Epoch [6/10], Loss: 0.0001
Epoch [7/10], Loss: 0.0004
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Test Accuracy: 85.45%
